In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [2]:
qq('start')

In [110]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

#from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections
import fasttext

In [4]:
df = pd.read_parquet('train_text.pq')

In [5]:
df.columns

Index(['vacancy_id', 'name', 'specializations', 'key_skills', 'employer',
       'groups_of_spec', 'processed_text', 'key_list', 'name_list'],
      dtype='object')

In [7]:
df.drop(columns=['groups_of_spec', 'key_list', 'name_list'], inplace=True)

In [10]:
df['text'] = df[['employer', 'name', 'key_skills', 'processed_text']].apply(' '.join, axis=1)

In [13]:
df.head(2)

,vacancy_id,specializations,text
4,2700005,"[137, 221, 296]",845fe6266ab1 разработчик python django неболь...
5,2700006,"[149, 242]",23ead4a8f164 торговый представитель работа тек...


In [12]:
df.drop(columns=['employer', 'name', 'key_skills', 'processed_text'], inplace=True)

In [17]:
def inf_vec(txt):
    return model.infer_vector(txt.split(' '))

In [19]:
df['text_vector'] = pqdm(df['processed_text'], inf_vec, n_jobs=10)

In [20]:
df.to_parquet('df_vector.pq')

In [14]:
def unpivot(obj):
    res = []
    for s in obj.specializations:
        new_obj = obj.to_dict()
        new_obj.pop('specializations')
        new_obj['specialization'] = s
        res.append(new_obj)
    return res

In [4]:
df=pd.read_parquet('df_vector.pq')

In [5]:
df.drop(columns=['processed_text', 'name', 'key_skills', 'groups_of_spec'], inplace = True)

In [6]:
df.head()

,vacancy_id,specializations,employer,key_list,name_list,text_vector
4,2700005,"[137, 221, 296]",845fe6266ab1,[],"[Разработчик, Python/django]","[-0.37388623, 0.26167643, 0.3595109, -0.166157..."
5,2700006,"[149, 242]",23ead4a8f164,"[работа, текущий, база, клиент, мерчендайзинг,...","[Торговый, представитель]","[-0.07712369, -0.11375963, 0.07628894, -0.0080..."
6,2700007,"[149, 196, 242, 324, 538]",9e3008e9f553,"[b2b, продажа, активный, продажа, развитие, пр...","[Менеджер, по, оптовым, продажам, тканей]","[-0.39096445, 0.33778182, -0.016544262, -0.177..."
8,2700010,"[398, 537, 587]",49f3280f53b6,[],"[Врач, клинической, лабораторной, диагностики,...","[-0.23469944, 0.35904798, 0.1611339, -0.071664..."
12,2700015,"[129, 157, 516]",24e6806caa88,[],[Швея-портной],"[-0.5796743, 0.25497875, 0.35563952, -0.068753..."


In [ ]:
df.to_parquet('df_vector.pq')

In [9]:
test_vac_id = set(dill.load(open('validatin.bin','rb')))

In [10]:
df['validation'] = df['vacancy_id'].apply(lambda x: x in test_vac_id)

In [15]:
res = []
for i, r in tqdm(df.iterrows(), total = df.shape[0]):
    res += unpivot(r)

In [115]:
df.head()

,vacancy_id,specializations,text,is_val,label
4,2700005,"[137, 221, 296]",845fe6266ab1 разработчик python django неболь...,False,__label__137 __label__221 __label__296
5,2700006,"[149, 242]",23ead4a8f164 торговый представитель работа тек...,False,__label__149 __label__242
6,2700007,"[149, 196, 242, 324, 538]",9e3008e9f553 менеджер оптовый продажа ткань b2...,False,__label__149 __label__196 __label__242 __label...
8,2700010,"[398, 537, 587]",49f3280f53b6 врач клинический лабораторный диа...,True,__label__398 __label__537 __label__587
12,2700015,"[129, 157, 516]",24e6806caa88 швея портной требоваться швея ла...,False,__label__129 __label__157 __label__516


In [114]:
df['label'] = df['specializations'].apply(lambda x: ' '.join(['__label__'+str(e) for e in x]))

In [118]:
train = df[~df['is_val']][['vacancy_id', 'label', 'text']].copy()

In [131]:
test = df[df['is_val']][['vacancy_id', 'label', 'text', 'specializations']].copy()

In [120]:
train.to_csv('train.txt', index=False, sep=' ', header=False,)

In [121]:
test.to_csv('test.txt', index=False, sep=' ', header=False,)

In [46]:
valid = set(dill.load(open('validatin.bin', 'rb')))

In [148]:
%%time
model = fasttext.train_supervised('train.txt', dim=300, epoch=10, minCount=2, ws=5, neg=10, verbose=3)

CPU times: user 3h 29min 42s, sys: 19.5 s, total: 3h 30min 2s
Wall time: 19min 22s


In [125]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [149]:
print_results(*model.test('train.txt'))

N	1172160
P@1	0.596
R@1	0.255


In [150]:
print_results(*model.test('test.txt'))

N	284164
P@1	0.570
R@1	0.243


In [153]:
p = 0.596
r = 0.255

In [154]:
2*p*r/(p+r)

0.35717978848413634

In [151]:
qq('done')

In [60]:
df['is_val'] = df['vacancy_id'].isin(valid)

In [62]:
test = df[df['is_val']]

In [36]:
test = df_unpivoted['is_val']

In [63]:
# test['text'] = test[['employer', 'name', 'key_skills', 'processed_text']].apply(' '.join, axis=1)

In [64]:
# test.drop(columns=['groups_of_spec', 'key_list', 'name_list', 'employer', 'name', 'key_skills', 'processed_text'], inplace=True)

In [65]:
test.head()

,vacancy_id,specializations,text,is_val
8,2700010,"[398, 537, 587]",49f3280f53b6 врач клинический лабораторный диа...,True
14,2700017,"[175, 181, 248, 313, 374, 580]",764a221a6ce7 администратор хозяйственный служб...,True
19,2700022,[193],8114330c1576 официант бармен обязанность обсл...,True
21,2700024,"[65, 66, 148, 149, 358, 535]",8632324c3155 менеджер продажа менеджер работа ...,True
22,2700025,"[9, 10, 82, 89, 172, 295]",ed4be96cf8ea оператор беспилотный летальный ап...,True


In [100]:
test.iloc[0].text

'49f3280f53b6 врач клинический лабораторный диагностика бактериология  компания kdl клинико диагностический лаборатория приглашать кандидат должность quot врач клинический лабораторный диагностика бактериология quot обязанность работа бактериологический отделение проведение бактериологический исследование требование образование высокий медицинский биологический переподготовка действующий сертификат бактериология либо клинический лабораторный диагностика знание уверенный пользователь условие работа стремительно развиваться компания лидер медицинский рынок официальный оформление сменный график работа quot белый quot заработный плата выплачиваться месяц задержка полный социальный пакет повышение квалификация счет компания возможность профессиональный рост корпоративный скидка услуга лаборатория сотрудник компания семья'

In [101]:
model.predict(test.iloc[0].text, k=6)

(('__label__398',
  '__label__587',
  '__label__131',
  '__label__128',
  '__label__185',
  '__label__155'),
 array([0.58572251, 0.17023057, 0.10375242, 0.04449557, 0.02447621,
        0.01403851]))

In [103]:
test = test.copy()

In [130]:
test.head()

,vacancy_id,label,text
8,2700010,__label__398 __label__537 __label__587,49f3280f53b6 врач клинический лабораторный диа...
14,2700017,__label__175 __label__181 __label__248 __label...,764a221a6ce7 администратор хозяйственный служб...
19,2700022,__label__193,8114330c1576 официант бармен обязанность обсл...
21,2700024,__label__65 __label__66 __label__148 __label__...,8632324c3155 менеджер продажа менеджер работа ...
22,2700025,__label__9 __label__10 __label__82 __label__89...,ed4be96cf8ea оператор беспилотный летальный ап...


In [155]:
%%time
test['predict'] = test['text'].apply(model.predict, k=6)

CPU times: user 2min 52s, sys: 324 ms, total: 2min 52s
Wall time: 2min 52s


In [156]:
test.head()

,vacancy_id,label,text,specializations,predict,pred
8,2700010,__label__398 __label__537 __label__587,49f3280f53b6 врач клинический лабораторный диа...,"[398, 537, 587]","((__label__398, __label__587"", __label__398"", ...","[398, 587, 398, 185, 155, 131]"
14,2700017,__label__175 __label__181 __label__248 __label...,764a221a6ce7 администратор хозяйственный служб...,"[175, 181, 248, 313, 374, 580]","((__label__248, __label__316, __label__255, __...","[248, 316, 255, 316, 329, 504]"
19,2700022,__label__193,8114330c1576 официант бармен обязанность обсл...,[193],"((__label__193, __label__329, __label__248, __...","[193, 329, 248, 518, 231, 193]"
21,2700024,__label__65 __label__66 __label__148 __label__...,8632324c3155 менеджер продажа менеджер работа ...,"[65, 66, 148, 149, 358, 535]","((__label__149, __label__196, __label__148, __...","[149, 196, 148, 242, 332, 538]"
22,2700025,__label__9 __label__10 __label__82 __label__89...,ed4be96cf8ea оператор беспилотный летальный ап...,"[9, 10, 82, 89, 172, 295]","((__label__47, __label__221, __label__245, __l...","[47, 221, 245, 142, 81, 273]"


In [157]:
test[test['vacancy_id']==2700017].predict.values

array([(('__label__248', '__label__316', '__label__255', '__label__316"', '__label__329', '__label__504"'), array([0.23508149, 0.12962542, 0.06665581, 0.05006903, 0.03730892,
       0.03699226]))], dtype=object)

In [158]:
%%time
test['pred']=test['predict'].apply(lambda x: [int(e.replace('__label__', '').replace('"','')) for e in x[0]])

CPU times: user 1.15 s, sys: 12 ms, total: 1.16 s
Wall time: 1.16 s


In [142]:
def f1score(y_true, y_pred):
    y_true = set(y_true)
    y_pred = set(y_pred)
    
    tp = len(y_true & y_pred)
    precision = tp / len(y_pred)
    recall = tp / len(y_true)
    if precision == 0.0 and recall == 0.0:
        score = 0.0
    else:
        score = 2 * precision * recall / (precision + recall)
    return score

def mean_f1score(y_true, y_pred):
    lst = []
    for vacancy_id, true_specs in y_true.items():
        lst.append(f1score(true_specs, y_pred[vacancy_id]))
    return np.mean(lst)

In [143]:
test.head(2)

,vacancy_id,label,text,specializations,predict,pred
8,2700010,__label__398 __label__537 __label__587,49f3280f53b6 врач клинический лабораторный диа...,"[398, 537, 587]","((__label__398, __label__587"", __label__398"", ...","[398, 587, 398, 185, 155, 131]"
14,2700017,__label__175 __label__181 __label__248 __label...,764a221a6ce7 администратор хозяйственный служб...,"[175, 181, 248, 313, 374, 580]","((__label__248, __label__316, __label__255, __...","[248, 316, 255, 316, 329, 504]"


In [144]:
y_true = test['specializations'].to_dict()

In [109]:
#test['pred'] = test['pred'].apply(lambda x: x[:3])

In [145]:
y_pred = test['pred'].to_dict()

In [146]:
mean_f1score(y_true, y_pred)

0.4250116148756022